In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import numpy as np

In [2]:



data = {
    "name": "Potomac Pizza",
    "address": "7777 Baltimore Ave, College Park, MD  20740, United States",
    "location": {
      "lat": 38.9873178,
      "lon": -76.9356036
    },
    "category": "Restaurant",
    "category2": "Italian food"
},
{
"name": "Wasabi Bistro Japanese Food & Bubble Tea",
"address": "4505 College Ave, College Park, MD  20740, United States",
"location": {
    "lat": 38.9818553,
    "lon": -76.9373256
},
"category": "Restaurant",
"category2": "Japanese food"
},
{
"name": "Q\u00f9 Japan",
"address": "7406 Baltimore Ave, College Park, MD  20740, United States",
"location": {
    "lat": 38.9811565,
    "lon": -76.9380815
},
"category": "Restaurant",
"category2": "Japanese food"
},
{
"name": "College Park Shopping Center",
"address": "7370 Baltimore Ave, College Park, MD  20740, United States",
"location": {
    "lat": 38.9806676,
    "lon": -76.9390872
},
"category": "Store",
"category2": "shopping"
},
df = pd.DataFrame(data)

In [3]:

user_profile = {'user_id': '66996b7b0025b402922b', 'cuisine': ['Japanese', 'Italian'], 'atmosphere': ['Cozy'], 'entertainment': ['Arcade', 'Parks'], 'socializing': 'BOTH', 'Time': ['MORNING'], 'shopping': 'YES', 'family_friendly': True, 'learning': ['Culture'], 'sports': ['Golf'], '$id': '66996b7b0025b402922b', '$tenant': '168138', '$createdAt': '2024-07-18T19:31:11.173+00:00', '$updatedAt': '2024-07-20T19:16:56.252+00:00', '$permissions': [], 'users': {'email': 'Nagelia.naman@gmail.com', 'uid': '66996b7b0025b402922b', 'firstName': 'Naman', 'lastName': 'Nagelia', 'address': None, 'premium': False, 'monthly_uses': 5, 'friends': ['66996d2e00314baa2a20', '669ea8a0002fd0a07a97', '66a850af002730eadb39', '66a05a77002bacd6dad0', '66a58e55001acebf9f51'], 'receivedRequests': [], 'sentRequests': ['669b45980030c00f3b8c', '669c735c001355ea24a7'], '$id': '66996b7b0025b402922b', '$tenant': '168138', '$createdAt': '2024-07-18T19:22:37.018+00:00', '$updatedAt': '2024-07-31T18:48:29.871+00:00', '$permissions': [], '$databaseId': '66930e1000087eb0d4bd', '$collectionId': '66930e5900107bc194dc'}, '$databaseId': '66930e1000087eb0d4bd', '$collectionId': '6696016b00117bbf6352'}


In [ ]:
le_type = LabelEncoder()
le_cuisine = LabelEncoder()
le_location = LabelEncoder()
le_price_range = LabelEncoder()
df['type'] = le_type.fit_transform(df['type'])
df['cuisine'] = le_cuisine.fit_transform(df['cuisine'])
df['location'] = le_location.fit_transform(df['location'])
df['price_range'] = le_price_range.fit_transform(df['price_range'])

In [ ]:
user_profile_encoded = {
    "type": le_type.transform([user_profile["type"]])[0],
    "cuisine": le_cuisine.transform([user_profile["cuisine"]])[0],
    "location": le_location.transform([user_profile["location"]])[0],
    "price_range": le_price_range.transform([user_profile["price_range"]])[0]
}

In [ ]:
features = ['category', 'category2', 'lat', 'lon']
# passed into model
X = df[features].values

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        # play around with this later
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        # one output for regression/classification models usually
        self.fc3 = nn.Linear(32, 1)
        
    # forward pass tells us how the data flows through the network
    def forward(self, x):
        # relu used to focus on positive numbers
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
        

In [ ]:
# retrieves # of features that will be in input
input_dim = X.shape[1]
# initialize model
model = SimpleNN(input_dim)
# checker of how well performance is
criterion = nn.MSELoss()
# optimizer to update weights of model to minimize loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
X_train_tensor = torch.FloatTensor(X_train, dtype=torch.float32)
X_test_tensor = torch.FloatTensor(X_test, dtype=torch.float32)

# training loop
epochs = 1000
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad
